In [1]:
# Use the following lines if you want to use Google Colab
# We presume you created a folder "i2dl" within your main drive folder, and put the exercise there.
# NOTE: terminate all other colab sessions that use GPU!

from google.colab import drive
import os

gdrive_path='/content/gdrive/MyDrive/shap-e'

# This will mount your google drive under 'MyDrive'
drive.mount('/content/gdrive', force_remount=True)
# In order to access the files in this notebook we have to navigate to the correct folder
os.chdir(gdrive_path)
# Check manually if all files are present
print(sorted(os.listdir()))

Mounted at /content/gdrive
['.git', '.gitignore', 'LICENSE', 'README.md', 'model-card.md', 'samples', 'samples.md', 'setup.py', 'shap_e', 'shap_e.egg-info', 'shap_e_model_cache']


In [2]:
!pip install -e .

Obtaining file:///content/gdrive/MyDrive/shap-e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-9cfr3ff8/clip_0c24120c46e04f08a5457d1a3ae429a6
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-9cfr3ff8/clip_0c24120c46e04f08a5457d1a3ae429a6
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.4 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=c61e6f5032ff7f7869a73b71ead38a75b6b7f152d7564cce2669ce311aa6e3b9
  Store

In [3]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.sample import latent_sds
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [5]:
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

/content/gdrive/MyDrive/shap-e/shap_e/models/nn/checkpoint.py:32: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, run_function, length, *args):
/content/gdrive/MyDrive/shap-e/shap_e/models/nn/checkpoint.py:44: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, *output_grads):
/content/gdrive/MyDrive/shap-e/shap_e/models/nn/checkpoint.py:62: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, run_function, length_1, length_2, *args):
/content/gdrive/MyDrive/shap-e/shap_e/models/nn/checkpoint.py:87: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, *all_output_gr

In [6]:
batch_size = 1
guidance_scale = 15.0
sigma_max = 160
sample_shape = (batch_size, model.d_latent)
latent = torch.randn(*sample_shape, device=device) * sigma_max
prompt = "a chair"

In [ ]:
alpha = 0.05
for i in range(10):
  output_latent = latent_sds(
      latent=latent,
      batch_size=batch_size,
      model=model,
      diffusion=diffusion,
      guidance_scale=guidance_scale,
      model_kwargs=dict(texts=[prompt] * batch_size),
      progress=True,
      clip_denoised=True,
      use_fp16=True,
      use_karras=True,
      karras_steps=64,
      sigma_min=1e-3,
      sigma_max=sigma_max,
      s_churn=0,
    )
  d_sds = latent - output_latent
  latent -= alpha * d_sds
  sds_loss = torch.mean(d_sds ** 2)
  print(f"SDS_loss in iteration {i + 1}: {sds_loss.item()}")

  0%|          | 0/64 [00:00<?, ?it/s]

SDS_loss in iteration 1: 20.49527359008789


  0%|          | 0/64 [00:00<?, ?it/s]

SDS_loss in iteration 2: 18.51491928100586


  0%|          | 0/64 [00:00<?, ?it/s]

SDS_loss in iteration 3: 16.735963821411133


  0%|          | 0/64 [00:00<?, ?it/s]

In [8]:
render_mode = 'nerf' # you can change this to 'stf'
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
#for i, latent in enumerate(latents):
#    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
#    display(gif_widget(images))

images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
display(gif_widget(images))
images = decode_latent_images(xm, output_latent, cameras, rendering_mode=render_mode)
display(gif_widget(images))

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAJ5sTZhlSZVkSo5nRpdhTpFiSpxgPZZgQJNgPo9gRY1gNJ1eRJ…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAJxhNJthNZthNJtgNJtgM5tgMpphNJpgNJpgM5lgNJlgM5lgMp…

In [ ]:
# Example of saving the latents as meshes.
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    t = decode_latent_mesh(xm, latent).tri_mesh()
    with open(f'example_mesh_{i}.ply', 'wb') as f:
        t.write_ply(f)
    with open(f'example_mesh_{i}.obj', 'w') as f:
        t.write_obj(f)